In [84]:
pip install pandas numpy tensorflow matplotlib opencv-python scikit-learn 


Note: you may need to restart the kernel to use updated packages.


In [85]:
import os
import cv2
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import random

from shutil import copyfile
from tensorflow.keras.layers import Conv2D,Add,MaxPooling2D, Dense, BatchNormalization,Input , Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [86]:
import os
import pandas as pd


# Define folder paths
source_folder = '/kaggle/input/draft-safety/train/train_safety'

df = pd.read_csv('/kaggle/input/draft-safety/train/train_annotation.csv')
print(df.head())




  image_id                       labels
0    img_0    person red_hat yellow_hat
1    img_1              person blue_hat
2    img_2       person vest yellow_hat
3    img_3  person white_hat yellow_hat
4    img_4    person red_hat yellow_hat


In [87]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Define your possible labels
possible_labels = ['person', 'vest', 'red_hat', 'blue_hat', 'white_hat', 'yellow_hat']

# Assuming your DataFrame has a column 'labels' with string labels, e.g. "person red_hat yellow_hat"
df['labels'] = df['labels'].apply(lambda x: x.split(' '))

# Create multi-hot encoded labels
mlb = MultiLabelBinarizer(classes=possible_labels)
encoded_labels = mlb.fit_transform(df['labels'])

# Create a new DataFrame from these encoded labels
encoded_df = pd.DataFrame(encoded_labels, columns=possible_labels)

# Concatenate these new columns with the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Optionally, check dtypes to ensure they're numeric
print(df.dtypes)


image_id      object
labels        object
person         int64
vest           int64
red_hat        int64
blue_hat       int64
white_hat      int64
yellow_hat     int64
dtype: object


In [88]:
import os

missing_images = [img for img in df['image_id'] if not os.path.exists('/kaggle/input/draft-safety/train/train_safety/' + img)]
print(f"Images manquantes : {len(missing_images)}")


Images manquantes : 1860


In [96]:
from PIL import Image

img_path = os.path.join('/kaggle/input/draft-safety/train/train_safety', df['image_id'].iloc[0])
try:
    img = Image.open(img_path)
    print("Image loaded successfully!")
except Exception as e:
    print("Error loading image:", e)


Error loading image: [Errno 2] No such file or directory: '/kaggle/input/draft-safety/train/train_safety/img_0.jpg.jpg.jpg'


In [100]:
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer

# Example: loading your CSV
df = pd.read_csv('/kaggle/input/draft-safety/train/train_annotation.csv')

# Split the labels (assuming they're space-separated)
df['labels'] = df['labels'].apply(lambda x: x.split())

# Define possible labels
possible_labels = ['person', 'vest', 'blue_hat', 'red_hat', 'white_hat', 'yellow_hat']

# Use MultiLabelBinarizer to create a binary matrix for the labels
mlb = MultiLabelBinarizer(classes=possible_labels)
encoded_labels = mlb.fit_transform(df['labels'])

# Create a DataFrame with the encoded labels
encoded_df = pd.DataFrame(encoded_labels, columns=possible_labels)

# Merge the new columns with the original DataFrame
df = pd.concat([df, encoded_df], axis=1)

# Check that the columns have been added
print(df.head())
print(df.columns)


  image_id                           labels  person  vest  blue_hat  red_hat  \
0    img_0    [person, red_hat, yellow_hat]       1     0         0        1   
1    img_1               [person, blue_hat]       1     0         1        0   
2    img_2       [person, vest, yellow_hat]       1     1         0        0   
3    img_3  [person, white_hat, yellow_hat]       1     0         0        0   
4    img_4    [person, red_hat, yellow_hat]       1     0         0        1   

   white_hat  yellow_hat  
0          0           1  
1          0           0  
2          0           1  
3          1           1  
4          0           1  
Index(['image_id', 'labels', 'person', 'vest', 'blue_hat', 'red_hat',
       'white_hat', 'yellow_hat'],
      dtype='object')


In [101]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

img_width, img_height = 224, 224  # adjust as needed

datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Ensure the image filenames include the proper extension
df['image_id'] = df['image_id'].apply(lambda x: x if x.endswith('.jpg') else x + '.jpg')

train_directory = '/kaggle/input/draft-safety/train/train_safety/'

train_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_directory,
    x_col='image_id',
    y_col=possible_labels,  # now these columns exist in the DataFrame
    target_size=(img_width, img_height),
    class_mode='raw',  # Use 'raw' for multi-label classification with numeric arrays
    batch_size=32,
    subset='training'
)

validation_generator = datagen.flow_from_dataframe(
    dataframe=df,
    directory=train_directory,
    x_col='image_id',
    y_col=possible_labels,
    target_size=(img_width, img_height),
    class_mode='raw',
    batch_size=32,
    subset='validation'
)


Found 1488 validated image filenames.
Found 372 validated image filenames.


In [102]:
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0

# Define your image dimensions
img_width, img_height = 224, 224  # or your preferred dimensions

# Load the base model without any weights
base_model = EfficientNetB0(input_shape=(img_width, img_height, 3), 
                             include_top=False, 
                             weights=None)
# Load the weights from the local file (adjust the path as necessary)
base_model.load_weights('/kaggle/input/weights/efficientnetb0_notop.h5')

# Freeze the base model if you want to use it for transfer learning
base_model.trainable = False

# Build your full model
possible_labels = ['person', 'vest', 'blue_hat', 'red_hat', 'white_hat', 'yellow_hat']
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(possible_labels), activation='sigmoid')  # Sigmoid for multi-label classification
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_4           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 128)                 │         163,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 6)                   │             774 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,214,313 (16.08 MB)

 Trainable params: 164,742 (643.52 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    steps_per_epoch=train_generator.samples // 32,
    validation_steps=validation_generator.samples // 32
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


46/46 ━━━━━━━━━━━━━━━━━━━━ 85s 2s/step - accuracy: 0.8802 - loss: 0.6040 - val_accuracy: 0.9886 - val_loss: 0.5179
Epoch 2/20
 1/46 ━━━━━━━━━━━━━━━━━━━━ 46s 1s/step - accuracy: 1.0000 - loss: 0.5363

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 1.0000 - loss: 0.5363 - val_accuracy: 1.0000 - val_loss: 0.4934
Epoch 3/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.9927 - loss: 0.5384 - val_accuracy: 0.9886 - val_loss: 0.5144
Epoch 4/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 1.0000 - loss: 0.5052 - val_accuracy: 1.0000 - val_loss: 0.5489
Epoch 5/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.9928 - loss: 0.5291 - val_accuracy: 0.9886 - val_loss: 0.5173
Epoch 6/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.9688 - loss: 0.5534 - val_accuracy: 1.0000 - val_loss: 0.5054
Epoch 7/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.9919 - loss: 0.5305 - val_accuracy: 0.9886 - val_loss: 0.5178
Epoch 8/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 1.0000 - loss: 0.4942 - val_accuracy: 1.0000 - val_loss: 0.4838
Epoch 9/20
46/46 ━━━━━━━━━━━━━━━━━━━━ 70s 1s/step - accuracy: 0.9931 - loss: 0.5262 - val_accuracy: 0.9886 - val_loss: 0.5141

In [ ]:
# Create a test ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)

# Adjust the directory path if needed; ensure it points to your test folder
test_directory = '/kaggle/input/draft-safety/test_safety/test_safety'

# Create the test generator
test_generator = test_datagen.flow_from_directory(
    directory=test_directory,
    target_size=(img_width, img_height),
    class_mode=None,    # No labels for test set
    batch_size=32,
    shuffle=False
)


In [ ]:
# Predict on the test set
predictions = model.predict(test_generator, verbose=1)

# Apply a threshold (commonly 0.5) to convert probabilities into binary predictions
predicted_binary = (predictions > 0.5).astype(int)


In [ ]:
# Convert binary predictions to label lists
predicted_labels_list = mlb.inverse_transform(predicted_binary)

# For each image, join the labels into a single string (space-separated, for example)
predicted_labels_str = [" ".join(labels) if labels else "none" for labels in predicted_labels_list]


In [ ]:
import pandas as pd

# Extract filenames from the test generator.
# Depending on how your test generator is set up, the filenames might include a folder path.
# You might need to clean the filename to extract just the image ID.
filenames = test_generator.filenames

# If the filenames include subdirectories, you can remove them.
# For example, if filenames look like 'test/img_0.jpg', you can extract the basename:
filenames = [fname.split('/')[-1] for fname in filenames]

# Create the submission DataFrame
submission_df = pd.DataFrame({
    'image_id': filenames,
    'labels': predicted_labels_str
})

# Check the first few rows of your submission file
print(submission_df.head())
# Save the submission file in the Kaggle output directory
submission_df.to_csv('/kaggle/working/submission.csv', index=False)
